# Basic Critique-Revise

This is an basic example of correcting an LLM's output using a pattern called critique-revise, where we highlight what part of the output is wrong and re-query the LLM for a correction.

In the below example, we define a Pydantic schema to match an array of tasks we want the LLM to compose for a given input. Take note that we expect the output tasks to match one of a defined set of types, and to return a valid ISO string as a due date:

In [1]:
# %pip install openai

from typing import Sequence
from enum import Enum
from datetime import datetime
from pydantic import BaseModel, Field, ValidationError

class TaskType(str, Enum):
    call = "Call"
    message = "Message"
    todo = "Todo"
    in_person_meeting = "In-Person Meeting"
    email = "Email"
    mail = "Mail"
    text = "Text"
    open_house = "Open House"

class Task(BaseModel):
    title: str = Field(..., description="The title of the tasks, reminders and alerts")
    due_date: datetime = Field(..., description="Due date. Must be a valid ISO date string with timezone")
    task_type: TaskType = Field(None, description="The type of task")
    
class Tasks(BaseModel):
    """JSON definition for creating tasks, reminders and alerts"""
    tasks: Sequence[Task]

We use this Pydantic schema with an OpenAI model as a function, and invoke it:

In [2]:
from langchain.chains.openai_functions import (
    convert_to_openai_function
)
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser


template = """Respond to the following user query to the best of your ability:

{query}""";

generate_prompt = ChatPromptTemplate.from_template(template)

user_query = "Set a reminder to renew our online property ads next week."

function_args = {"functions": [convert_to_openai_function(Tasks)]}

task_function_call_model = ChatOpenAI(model="gpt-3.5-turbo").bind(**function_args)

output_parser = JsonOutputFunctionsParser()

In [3]:
# const traceGroup = new TraceGroup("CritiqueReviseChain");
# const groupManager = await traceGroup.start();

chain = generate_prompt | task_function_call_model | output_parser

result = chain.invoke({ "query": user_query })

print(result)

{'tasks': [{'title': 'Renew online property ads', 'due_date': 'next week', 'task_type': 'Reminder'}]}


We can see that despite the passed function schema, the model was influenced by the particular user input, and the the outputted task type does not match one of the defined types. We use our original Pydantic model's `validate` method in a wrapped validator method to show this. We extract the error message summarizing the problem:

In [4]:
def output_validator(output):
    try:
        Tasks.validate(output)
    except ValidationError as e:
        return (False, str(e))
    
    return [True]

output_validator(result)

(False,
 "2 validation errors for Tasks\ntasks -> 0 -> due_date\n  invalid datetime format (type=value_error.datetime)\ntasks -> 0 -> task_type\n  value is not a valid enumeration member; permitted: 'Call', 'Message', 'Todo', 'In-Person Meeting', 'Email', 'Mail', 'Text', 'Open House' (type=type_error.enum; enum_values=[<TaskType.call: 'Call'>, <TaskType.message: 'Message'>, <TaskType.todo: 'Todo'>, <TaskType.in_person_meeting: 'In-Person Meeting'>, <TaskType.email: 'Email'>, <TaskType.mail: 'Mail'>, <TaskType.text: 'Text'>, <TaskType.open_house: 'Open House'>])")

Now, we define a revise chain that will attempt to fix the problem. We reuse the previously defined model with the bound function call arguments:

In [5]:
revise_template = """Original prompt:
--------------
{original_prompt}
--------------

Completion:
--------------
{completion}
--------------

Above, the completion did not satisfy the constraints given by the original prompt and provided schema.

Error:
--------------
{error}
--------------

Try again. Only respond with an answer that satisfies the constraints laid out in the original prompt and provided schema:"""

revise_prompt = ChatPromptTemplate.from_template(revise_template)

revise_chain = (revise_prompt | task_function_call_model | JsonOutputFunctionsParser()).with_config(run_name="ReviseChain")

And finally, we rerun the erroneous output with the formatted original prompt, completion, and error message until it passes the validation successfully:

In [6]:
from langchain.callbacks.manager import (
    trace_as_chain_group,
)
import json

with trace_as_chain_group("CritiqueRevise", inputs={"query": user_query}) as group_manager:
    fix_count = 0
    formatted_original_prompt = generate_prompt.format(query=user_query)
    result = chain.invoke({ "query": user_query }, config={"callbacks": group_manager})
    validator_result = output_validator(result)
    while validator_result[0] == False and fix_count < 5:
        result = revise_chain.invoke({
            "original_prompt": formatted_original_prompt,
            "completion": json.dumps(result),
            "error": json.dumps(validator_result[1])
        }, config={"callbacks": group_manager})
        print(result)
        validator_result = output_validator(result)
        fix_count += 1

{'tasks': [{'title': 'Renew online property ads', 'due_date': '2022-12-12', 'task_type': 'Reminder'}]}
{'tasks': [{'title': 'Renew online property ads', 'due_date': '2022-12-12', 'task_type': 'Reminder'}]}
{'tasks': [{'title': 'Renew online property ads', 'due_date': '2022-12-12', 'task_type': 'Todo'}]}
{'tasks': [{'title': 'Renew online property ads', 'due_date': '2022-12-12T12:00:00Z', 'task_type': 'Todo'}]}


And the final result matches our original Pydantic schema:

In [7]:
result

{'tasks': [{'title': 'Renew online property ads',
   'due_date': '2022-12-12T12:00:00Z',
   'task_type': 'Todo'}]}

You can peruse the following LangSmith trace showing it in action:

https://smith.langchain.com/public/7e554bfa-2ed8-4d10-a06a-396c82f22820/r